## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\Ben\Anaconda3\envs\cvdl\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [9]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), input_shape=(32,32,3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3,3)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_9 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
batch_normalization_10 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
flatten_4 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               1081700   
__________

C:\Users\Ben\Anaconda3\envs\cvdl\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


In [10]:
classifier.fit(x_train, y_train, batch_size=100, epochs=30)

Epoch 1/30
50000/50000 [==============================] - 66s 1ms/step - loss: 1.5005 - acc: 0.5237
Epoch 2/30
50000/50000 [==============================] - 68s 1ms/step - loss: 0.9260 - acc: 0.6776
Epoch 3/30
50000/50000 [==============================] - 65s 1ms/step - loss: 0.7319 - acc: 0.7434
Epoch 4/30
50000/50000 [==============================] - 68s 1ms/step - loss: 0.5889 - acc: 0.7943
Epoch 5/30
50000/50000 [==============================] - 66s 1ms/step - loss: 0.4673 - acc: 0.8350
Epoch 6/30
50000/50000 [==============================] - 64s 1ms/step - loss: 0.3671 - acc: 0.8701
Epoch 7/30
50000/50000 [==============================] - 104s 2ms/step - loss: 0.3056 - acc: 0.8938
Epoch 8/30
50000/50000 [==============================] - 104s 2ms/step - loss: 0.2418 - acc: 0.9160
Epoch 9/30
50000/50000 [==============================] - 104s 2ms/step - loss: 0.1983 - acc: 0.9305
Epoch 10/30
50000/50000 [==============================] - 105s 2ms/step - loss: 0.1735 - acc: 0.

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [11]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[6.4984437e-05, 6.2138966e-10, 1.2594148e-08, 9.9824250e-01,
        1.6800754e-03, 1.2420665e-05, 4.2031951e-09, 4.5692380e-10,
        5.9263788e-10, 7.9012765e-14]], dtype=float32)